In [15]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

def feature_normalize(train_X):

    global mean, std
    mean = np.mean(train_X, axis=0)
    std = np.std(train_X, axis=0)

    return (train_X - mean) / std

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)
m, n = housing.data.shape

train_num = int(m*0.9)
print("Train numbers :", train_num)

X = housing.data
y = housing.target
y = y.reshape(-1,1)
###### Implement Data Preprocess here ######
trainX = X[:train_num]
trainY = y[:train_num]
trainX = feature_normalize(trainX)

testX = X[train_num:]
testY = y[train_num:]
testX = feature_normalize(testX)

inputX = tf.placeholder(tf.float32, [None, n])
inputY = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([n,1]))
b = tf.Variable(tf.zeros([1]))

#y = Wx + b
learning_rate = 0.01
Y = tf.add(tf.matmul(inputX, W), b)
loss = tf.reduce_mean(tf.abs(Y - inputY) / inputY)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss) 


init = tf.global_variables_initializer()
training_epochs = 200

###### Start TF session ######
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        #print epoch
        sess.run(optimizer, feed_dict={ inputX: trainX, inputY: trainY})
        c = sess.run(loss, feed_dict={inputX: testX, inputY: testY})
        #print c
    
    training_cost = sess.run(loss, feed_dict={inputX: trainX, inputY: trainY})
    print("Training cost=", training_cost, '\n')
    
    testing_cost = sess.run(loss, feed_dict={inputX: testX, inputY: testY})
    weight = sess.run(W)
    bias = sess.run(b)
    print("Testing cost=", testing_cost, '\n')
    
    predY = np.matmul(testX, weight) + bias
    error_rate = tf.reduce_mean(tf.abs(testY - predY) / testY)
    print("Testing cost=", error_rate.eval(), '\n')
    show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######

('Shape of dataset:', (20640, 8))
('Shape of label:', (20640,))
('Train numbers :', 18576)
('Training cost=', 0.28782779, '\n')
('Testing cost=', 0.25598106, '\n')
('Testing cost=', 0.25598107457767511, '\n')
